In [1]:
import pandas as pd
import numpy as np
import os
import re
import datetime

In [2]:
def getDate(filename):
    match_str = re.search(r'\d{4}-\d{2}-\d{2}',filename)
    try:
        return datetime.datetime.strptime(match_str.group(), '%Y-%m-%d').date()
    except ValueError:
        return None

In [3]:
def cleanColumn(name):
    name = name.upper()
    clean_name = re.sub(r'[^a-zA-Z0-9_]', '', name)
    clean_name = clean_name.replace('\n', ' ')
    if clean_name == 'SHELTERNFI':
        clean_name = 'SHELTER'
    if clean_name == 'PROTECTIONCP':
        clean_name = 'PC_CP'
    if clean_name == 'PROTECTIONGBV':
        clean_name = 'PC_GBV'
    if clean_name == 'PROTECTIONMA':
        clean_name = 'PC_MA'
    if clean_name == 'TOTALREACHED':
        clean_name = 'PEOPLEREACHED'
    return clean_name

In [4]:
#using csv and xlsx
csv_dir = 'ukraine_data_excel'


In [92]:
#using xlsx
sheet1_df = []
sheet2_df = []
sheet3_df = []

In [93]:
#using xlsx
for filename in os.listdir(csv_dir):
  if filename.endswith('.xlsx'):
    file_path = os.path.join(csv_dir,filename)
    excel_df = pd.ExcelFile(file_path)
    files_date = getDate(filename)
    for sheet in excel_df.sheet_names:
      df = pd.read_excel(excel_df,sheet_name=sheet)
      df.columns = [cleanColumn(col) for col in df.columns]
      df['DATE'] = files_date
      if sheet == 'Num_of_Orgs_by_Oblast':
        sheet1_df.append(df)
      elif sheet == 'People_Reached_by_Oblast':
        sheet2_df.append(df)
      elif sheet == 'UDE_Inputs':
        sheet3_df.append(df)

In [94]:
combined_sheet1 = pd.concat(sheet1_df, keys=[str(df['DATE'].iloc[0]) for df in sheet1_df])
res_sheet1 = combined_sheet1.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()
merged_sheet1 = pd.merge(res_sheet1, combined_sheet1[['ADMIN1_ID', 'DATE', 'OBLAST']], on=['ADMIN1_ID', 'DATE'], how='left')

C:\Users\rober\AppData\Local\Temp\ipykernel_22852\3247654741.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res_sheet1 = combined_sheet1.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()


In [95]:
combined_sheet2 = pd.concat(sheet2_df, keys=[str(df['DATE'].iloc[0]) for df in sheet2_df])
res_sheet2 = combined_sheet2.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()
merged_sheet2 = pd.merge(res_sheet2, combined_sheet2[['ADMIN1_ID', 'DATE', 'OBLAST']], on=['ADMIN1_ID', 'DATE'], how='left')

C:\Users\rober\AppData\Local\Temp\ipykernel_22852\3947036614.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res_sheet2 = combined_sheet2.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()


In [96]:
combined_sheet3 = pd.concat(sheet3_df, keys=[str(df['DATE'].iloc[0]) for df in sheet3_df])
res_sheet3 = combined_sheet3.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()
merged_sheet3 = pd.merge(res_sheet3, combined_sheet3[['ADMIN1_ID', 'DATE', 'OBLAST']], on=['ADMIN1_ID', 'DATE'], how='left')

C:\Users\rober\AppData\Local\Temp\ipykernel_22852\2370811461.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res_sheet3 = combined_sheet3.groupby(['ADMIN1_ID', 'DATE']).sum().reset_index()


In [97]:
merged_sheet1['PEOPLE_REACHED'] = merged_sheet3['PEOPLEREACHED']

In [101]:
#Num_of_Orgs_by_Oblast
merged_sheet1.head(35)


,ADMIN1_ID,DATE,CCCM,CCS,ETC,EDUCATION,FSL,HEALTH,MPC,NUTRITION,...,PC_CP,PC_GBV,PC_MA,PC_PC,SHELTER,WASH,TOTAL,LOGISTICS,OBLAST,PEOPLE_REACHED
0,UA05,2022-04-28,2.0,1.0,1.0,5.0,17.0,16.0,10.0,0.0,...,2.0,2.0,2.0,11.0,6.0,7.0,42.0,0.0,Vinnytska,91200
1,UA05,2022-05-06,2.0,1.0,1.0,5.0,20.0,17.0,10.0,0.0,...,2.0,3.0,2.0,11.0,6.0,8.0,45.0,0.0,Vinnytska,113000
2,UA05,2022-05-12,2.0,1.0,1.0,6.0,20.0,18.0,10.0,0.0,...,4.0,4.0,2.0,14.0,6.0,9.0,49.0,0.0,Vinnytska,138800
3,UA05,2022-05-19,2.0,1.0,1.0,6.0,20.0,20.0,10.0,0.0,...,6.0,4.0,2.0,14.0,6.0,9.0,50.0,0.0,Vinnytska,140600
4,UA05,2022-05-26,0.0,1.0,1.0,6.0,20.0,20.0,9.0,0.0,...,6.0,4.0,4.0,12.0,7.0,9.0,48.0,0.0,Vinnytska,216600
5,UA05,2022-06-02,0.0,1.0,1.0,6.0,23.0,20.0,11.0,0.0,...,8.0,4.0,3.0,0.0,7.0,9.0,58.0,0.0,Vinnytska,236000
6,UA05,2022-06-09,1.0,1.0,1.0,6.0,23.0,20.0,11.0,0.0,...,8.0,4.0,3.0,0.0,8.0,9.0,58.0,0.0,Vinnytska,240100
7,UA05,2022-06-16,1.0,1.0,1.0,6.0,24.0,20.0,13.0,0.0,...,10.0,5.0,3.0,0.0,9.0,11.0,59.0,0.0,Vinnytska,250600
8,UA05,2022-06-23,1.0,1.0,1.0,6.0,24.0,20.0,13.0,0.0,...,10.0,5.0,3.0,0.0,9.0,11.0,59.0,0.0,Vinnytska,250600
9,UA05,2022-06-30,1.0,1.0,1.0,6.0,22.0,20.0,12.0,0.0,...,10.0,5.0,3.0,0.0,10.0,13.0,56.0,0.0,Vinnytska,210500


In [ ]:
merged_sheet1['OBLAST'].value_counts()

In [13]:
sum(merged_sheet3['PEOPLEREACHED'].value_counts())

750

In [99]:
merged_sheet1['PEOPLE_REACHED'] = merged_sheet3['PEOPLEREACHED']

In [100]:
# replace zero values in the empty tuple to NAN for future linear interpolation
#NOTE : THIS IS HARD CODED NOT OPTIMAL
merged_sheet1.iloc[25] = merged_sheet1.iloc[25].replace(0,np.nan)
merged_sheet1.iloc[26] = merged_sheet1.iloc[26].replace(0,np.nan)

In [102]:
#check for null values
merged_sheet1.isna().any().any()

True

In [103]:
# check for null values in specific row
merged_sheet1.iloc[26].isna().any()

True

In [104]:
#filled null values using linear interpolation
interpolated_df = merged_sheet1.interpolate(method='linear')

In [105]:
interpolated_df.isna().any().any()

False

In [108]:
# rounded all int and float dtypes since interpolation caused floats
interpolated_df = interpolated_df.applymap(lambda x: round(x, 1) if isinstance(x, (int, float)) else x)

In [ ]:
interpolated_df.head(35)

In [16]:
merged_sheet1.shape

(750, 21)

In [17]:
#People_Reached_by_Oblast
merged_sheet2.head(10)

,ADMIN1_ID,DATE,FSL,GENERALPROTECTION,CP,MINEACTION,PROTECTIONTOTAL,HEALTH,MPC,SHELTER,WASH,PEOPLEREACHED,OBLAST
0,UA05,2022-04-28,7900.0,11200.0,100.0,500.0,11900.0,4600.0,91200.0,15800.0,2300.0,91200.0,Vinnytska
1,UA05,2022-05-06,14300.0,9600.0,100.0,500.0,14300.0,4600.0,113000.0,16200.0,2300.0,113000.0,Vinnytska
2,UA05,2022-05-12,15500.0,11400.0,200.0,500.0,16300.0,9900.0,138800.0,17500.0,3700.0,138800.0,Vinnytska
3,UA05,2022-05-19,17500.0,0.0,500.0,500.0,20300.0,23200.0,140600.0,17500.0,3700.0,140600.0,Vinnytska
4,UA05,2022-05-26,17600.0,0.0,4400.0,700.0,25600.0,23300.0,216600.0,21700.0,3800.0,216600.0,Vinnytska
5,UA05,2022-06-02,19000.0,0.0,21500.0,700.0,48000.0,23300.0,236000.0,33300.0,3800.0,236000.0,Vinnytska
6,UA05,2022-06-09,19000.0,0.0,21500.0,700.0,21000.0,23300.0,240100.0,40500.0,23800.0,240100.0,Vinnytska
7,UA05,2022-07-07,39200.0,0.0,46500.0,11800.0,94900.0,27500.0,214600.0,59400.0,42700.0,214600.0,Vinnytska
8,UA05,2022-07-14,39200.0,0.0,35700.0,11800.0,75700.0,27500.0,216000.0,60700.0,42700.0,216000.0,Vinnytska
9,UA05,2022-07-21,39200.0,0.0,35700.0,99800.0,174700.0,39000.0,217200.0,62600.0,42700.0,217200.0,Vinnytska


In [18]:
#UDE_Inputs
merged_sheet3.head()

,ADMIN1_ID,DATE,PEOPLEREACHED,NUMBEROFORGANISATIONS,OBLAST
0,UA05,2022-04-28,91200,42,Vinnytska
1,UA05,2022-05-06,113000,45,Vinnytska
2,UA05,2022-05-12,138800,49,Vinnytska
3,UA05,2022-05-19,140600,50,Vinnytska
4,UA05,2022-05-26,216600,48,Vinnytska


In [19]:
# copy_df = pd.DataFrame(np.nan, index=merged_sheet1.index,columns=merged_sheet1.columns)
# copy_df['DATE'] = merged_sheet1['DATE']
# copy_df['OBLAST'] = merged_sheet1['OBLAST']
# copy_df.head()

In [20]:
# # Create an Excel writer object
# excel_writer1 = pd.ExcelWriter('Data_ Round 33 - Ukraine 5W - 2022-11-10.xlsx', engine='xlsxwriter')
# excel_writer2 = pd.ExcelWriter('Data_ Round 34 - Ukraine 5W - 2022-11-25.xlsx', engine='xlsxwriter')
# # Write each DataFrame to a separate sheet in the Excel file
# copy_df.to_excel(excel_writer1, sheet_name='Num_of_Orgs_by_Oblast', index=False)
# copy_df.to_excel(excel_writer2, sheet_name='Num_of_Orgs_by_Oblast', index=False)

In [21]:
merged_sheet1.columns

Index(['ADMIN1_ID', 'DATE', 'CCCM', 'CCS', 'ETC', 'EDUCATION', 'FSL', 'HEALTH',
       'MPC', 'NUTRITION', 'PROTECTIONTOTAL', 'PC_CP', 'PC_GBV', 'PC_MA',
       'PC_PC', 'SHELTER', 'WASH', 'TOTAL', 'LOGISTICS', 'OBLAST',
       'PEOPLE_REACHED'],
      dtype='object')

In [22]:
merged_sheet1.shape

(750, 21)

In [23]:
merged_sheet3.shape

(750, 5)

In [26]:
merged_sheet1.DATE.value_counts()

2022-09-29    50
2022-04-28    25
2022-08-11    25
2022-12-22    25
2022-12-08    25
2022-11-25    25
2022-11-10    25
2022-10-27    25
2022-10-13    25
2022-09-22    25
2022-09-15    25
2022-09-08    25
2022-08-25    25
2022-08-18    25
2022-08-04    25
2022-05-06    25
2022-07-28    25
2022-07-21    25
2022-07-14    25
2022-07-07    25
2022-06-30    25
2022-06-23    25
2022-06-16    25
2022-06-09    25
2022-06-02    25
2022-05-26    25
2022-05-19    25
2022-05-12    25
2023-01-05    25
Name: DATE, dtype: int64